In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['axes.unicode_minus'] = False


### 1. 
#### A, B, C 동별 입주민의 주민대표 찬반 투표결과. 동별 찬반 비율이 동일한지 귀무가설, 대립가설 세우고 검정통계량 검정
- [찬성] - A:50, B:60, C:65
- [반대] - A:45, B:32, C:55

In [15]:
# 지금 이 상황을 어떤 상황으로 봐야 할까? A, B, C 3개의 찬/반 비율이 같은지 검정해야 한다. 이 3개의 값은 같은 값인가?
# 그러면 비모수검정: 명목척도: 독립표본 3: 동일성 검정으로 본다.

# H0 : A, B, C 3개 집단의 찬/반 비율 차이는 없다.
# H1 : A, B, C 3개 집단의 찬/반 비율 차이가 있다.

data = pd.DataFrame(data={'A':[50, 45], 'B':[60, 32], 'C':[65, 55]}, index=['찬성', '반대'])
data

from scipy.stats import chi2_contingency
chistat, p, df, expec = chi2_contingency(data, correction=False)

print('검정통계량은 {:.3f}이고 유의확률 p-value는 {:.3f} 이다.'.format(chistat, p))
print('유의확률이 {:.3f}이므로 귀무가설을 기각할 수 없다.'.format(p))

검정통계량은 3.667이고 유의확률 p-value는 0.160 이다.
유의확률이 0.160이므로 귀무가설을 기각할 수 없다.
